In [1]:
!pip install -r requirements.txt
!pip install jupyter-lsp
!pip install 'python-lsp-server[all]'
!pip install "pymongo[srv]"
!%config Completer.use_jedi = False


ERROR: Ignored the following versions that require a different python version: 1.22.0 Requires-Python >=3.8; 1.22.0rc1 Requires-Python >=3.8; 1.22.0rc2 Requires-Python >=3.8; 1.22.0rc3 Requires-Python >=3.8; 1.22.1 Requires-Python >=3.8; 1.22.2 Requires-Python >=3.8; 1.22.3 Requires-Python >=3.8; 1.22.4 Requires-Python >=3.8; 1.23.0 Requires-Python >=3.8; 1.23.0rc1 Requires-Python >=3.8; 1.23.0rc2 Requires-Python >=3.8; 1.23.0rc3 Requires-Python >=3.8; 1.23.1 Requires-Python >=3.8; 1.23.2 Requires-Python >=3.8; 1.23.3 Requires-Python >=3.8; 1.23.4 Requires-Python >=3.8; 1.23.5 Requires-Python >=3.8; 1.24.0rc1 Requires-Python >=3.8
ERROR: Could not find a version that satisfies the requirement numpy==1.23.4 (from versions: 1.3.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.9.0, 1.9.1, 1.9.2, 1.9.3, 1.10.0.post2, 1.10.1, 1.10.2, 1.10.4, 1.11.0, 1.11.1, 1.11.2, 1.11.3, 1.12.0, 1.12.1, 1.13.0rc1, 1.13.0rc2, 1.13.0, 1.13.1, 1.13.3, 1.14.0rc1, 1.14.0

In [2]:
import numpy as np
import pandas as pd

from persistence import Connection as Database

TypeError: 'type' object is not subscriptable

In [3]:
from typing import List, Tuple

In [4]:
# load some data 
db: Database = Database()

Connected to DB


In [5]:
resumes = db.fetch_all_unlabeled_resumes()

In [6]:
import importlib
from model import keyword_utils
importlib.reload(keyword_utils)

<module 'model.keyword_utils' from '/home/jupyter/OpenCV/root/backend/model/keyword_utils.py'>

In [7]:
from model import Job as J

importlib.reload(J)
Job = J.Job

In [8]:
print(len(resumes))

1


# Observations
- We observed that it/IT is a conflict with stop-word removal …
- Take the length of the experience into account 
- For example, statistical and statsitcs are close, maybe relevant to use embeddnings istead of intersection

In [9]:
print(len(resumes))

print(resumes[1]['interests'])

1


IndexError: list index out of range

In [ ]:

print(keyword_utils.get_section_texts_with_score_from_resume(resumes[0]))

In [ ]:
from model.train_dictionaries import train_keywords_for_job_title
j = train_keywords_for_job_title("Data scientist", resumes)

In [ ]:
j.keywords

for k in j.keywords.items(): 
    print(k)
    break
    
# print("\n".join([f"{k},{v}" for k, v in j.keywords.items()]))

In [ ]:
from persistence.cloud_storage.cloud_storage import write_job_keywords 

write_job_keywords(j)

## Need to fix
 - Training keyword/weights Automatically

## Training
all cvs -> split into labeled-parts -> extract keywords (keybert) -> update score with label -> for each word output average -> List of words with score for position 


## Other features
 - Experience duration? 
 - How long ago? 

In [ ]:
resumes[1]['contactInfo']

In [ ]:
import model
from prediction import prediction
from util import json_to_df 
importlib.reload(prediction)
importlib.reload(model.resume_scorer)
importlib.reload(json_to_df)


improved = prediction.improve_cv("Data scientist", resumes[1])

In [15]:
improved

{'title': 'Data Scientist at Thales',
 '_id': ObjectId('6374781d09ad88efd5044c9c'),
 'summary': {'value': '', 'label': 0.0},
 'experience': {'value': [{'value': {'company': 'Fondazione Santa Lucia',
     'title': 'Engineering Trainee',
     'location': '',
     'fromDate': 'Mars 2008',
     'toDate': 'September 2008',
     'description': 'Linear Inverse techniques for high resolution EEG and brain hyper- connectivity estimation in healthy subjects'},
    'label': 0.13508663678798602},
   {'value': {'company': 'Fondazione Santa Lucia, Neuroeletrical Imaging and Brain Computer Interface Laboratory',
     'title': 'Term Contract',
     'location': 'Rome',
     'fromDate': 'October 2008',
     'toDate': 'December 2008',
     'description': 'Application of high resolution EEG techniques and multivariate models for the brain connectivity estimation in healthy and patient subjects.'},
    'label': 0.1385201750091479},
   {'value': {'company': 'Thales',
     'title': 'Data Scientist',
     'lo

In [16]:
importlib.reload(json_to_df)
from jsondiff import diff
res = resumes[1]

df, md = json_to_df.create_df(res)

print(df[df['type'] == 'none'].to_dict('records'))

back = json_to_df.get_resume_dict_from_dataframe(df, md)

print(df[df['type'] == 'contactInfo'])
print(res['contactInfo'])
print(back['contactInfo'])
print(diff(res, back))

[]
  value  label         type                                address  \
1   NaN    0.0  contactInfo  Abraham Lincoln Street 1, Chicago, IL   

          website               linkedin          name phoneNumber  \
1  my-website.com  linkedin.com/in/me123  Monica Music    01012345   

                email          github  ... company title location fromDate  \
1  monica@music.co.kr  monicamusic123  ...     NaN   NaN      NaN      NaN   

  toDate description institution proficiency level  date  
1    NaN         NaN         NaN         NaN   NaN   NaN  

[1 rows x 22 columns]
  value  label         type                                address  \
1   NaN    0.0  contactInfo  Abraham Lincoln Street 1, Chicago, IL   

          website               linkedin          name phoneNumber  \
1  my-website.com  linkedin.com/in/me123  Monica Music    01012345   

                email          github  ... company title location fromDate  \
1  monica@music.co.kr  monicamusic123  ...     NaN   NaN 

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
